<a href="https://colab.research.google.com/github/sai2119/Tensorflow2.0/blob/master/audio_book_deep_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



THE BUSINESS CASE ACTION PLAN











1 preprocess the data 
*  balance the data set
* divie the dataset in training,validation and test
*  save the data in a tensor friendly format(npz)

2 create the ML algorithm

---------------------------------------------------------------------------------------------------

LOADING DATA

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Audiobooks_data.csv to Audiobooks_data.csv


In [2]:
import io
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import preprocessing
import seaborn as sns
sns.set()
data =  np.loadtxt(io.BytesIO(uploaded['Audiobooks_data.csv']),delimiter=',')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
unscaled_input_all = data[:,1:-1]
targets_all = data[:,-1]

Balance the dataset

In [0]:
num_one_targets = int(np.sum(targets_all))
num_one_targets
zero_target_counter = 0
indices_to_remove= []
for i in range(targets_all.shape[0]):
  if targets_all[i] == 0:
    zero_target_counter += 1
    if zero_target_counter > num_one_targets:
      indices_to_remove.append(i)
unscaled_inputs_equal_prior = np.delete(unscaled_input_all, indices_to_remove, axis= 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis =0)


Standardize the inputs


In [0]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_prior)

shuffle the data


In [0]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

Split the dataset into train validation and

In [18]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1788.0 3579 0.49958088851634536
239.0 447 0.5346756152125279
210.0 448 0.46875


 Save the three datasets in *.npz

In [0]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [0]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('Audiobooks_data_train.npz')

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(np.float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(np.int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

In [21]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Epoch 1/100
36/36 - 0s - loss: 0.5117 - accuracy: 0.7849 - val_loss: 0.3932 - val_accuracy: 0.8658
Epoch 2/100
36/36 - 0s - loss: 0.3533 - accuracy: 0.8782 - val_loss: 0.3115 - val_accuracy: 0.8837
Epoch 3/100
36/36 - 0s - loss: 0.3095 - accuracy: 0.8863 - val_loss: 0.2907 - val_accuracy: 0.8837
Epoch 4/100
36/36 - 0s - loss: 0.2907 - accuracy: 0.8930 - val_loss: 0.2826 - val_accuracy: 0.8881
Epoch 5/100
36/36 - 0s - loss: 0.2807 - accuracy: 0.8938 - val_loss: 0.2741 - val_accuracy: 0.8904
Epoch 6/100
36/36 - 0s - loss: 0.2696 - accuracy: 0.8983 - val_loss: 0.2671 - val_accuracy: 0.8926
Epoch 7/100
36/36 - 0s - loss: 0.2646 - accuracy: 0.8997 - val_loss: 0.2722 - val_accuracy: 0.8949
Epoch 8/100
36/36 - 0s - loss: 0.2599 - accuracy: 0.9019 - val_loss: 0.2622 - val_accuracy: 0.8949
Epoch 9/100
36/36 - 0s - loss: 0.2550 - accuracy: 0.9039 - val_loss: 0.2611 - val_accuracy: 0.8971
Epoch 10/100
36/36 - 0s - loss: 0.2528 - accuracy: 0.9028 - val_loss: 0.2545 - val_accuracy: 0.8949
Epoch 11/

In [24]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 1ms/step - loss: 0.2257 - accuracy: 0.9219


In [25]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.23. Test accuracy: 92.19%
